#  Name: M.Musadiq
# Reg No. SP23-BCS-095
# Section:C
# Lab Assignment: 5


## Step-by-Step Tasks

### Task 1: Vector Initialization
- Create a large 1D vector **A** of size **N = 10,000,000** on the root process.  
- Initialize it with values `A[i] = i + 1`.

### Task 2: Data Distribution
- Divide **A** into equal sub-vectors and distribute them among all processes using:
  - `MPI_Scatter()`
- Each process receives its sub-array `local_A`.

### Task 3: Local Computation
- Each process computes the partial sum of its sub-vector:
  ```c
  double local_sum = 0;
  for (int i = 0; i < local_size; i++)
      local_sum += local_A[i];


In [20]:
!apt-get update -qq
!apt-get install -y mpich > /dev/null


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [17]:
%%writefile distributed_sum_full.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[]) {
    int rank, size;
    long N = 10000000;
    double *A = NULL;
    double local_sum = 0.0, global_sum = 0.0, avg_sum = 0.0;
    double start_time, end_time, parallel_time, serial_time;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    long local_n = N / size;
    long remainder = N % size;
    long *sendcounts = NULL, *displs = NULL;

    if (rank == 0) {
        sendcounts = (long*)malloc(size * sizeof(long));
        displs = (long*)malloc(size * sizeof(long));
        long offset = 0;
        for (int i = 0; i < size; i++) {
            sendcounts[i] = local_n + (i < remainder ? 1 : 0);
            displs[i] = offset;
            offset += sendcounts[i];
        }

        A = (double*)malloc(N * sizeof(double));
        for (long i = 0; i < N; i++)
            A[i] = i + 1;
    }

    long my_local_n;
    if (rank == 0) my_local_n = sendcounts[0];
    MPI_Scatter(sendcounts, 1, MPI_LONG, &my_local_n, 1, MPI_LONG, 0, MPI_COMM_WORLD);

    double *local_A = (double*)malloc(my_local_n * sizeof(double));

    start_time = MPI_Wtime();
    MPI_Scatterv(A, sendcounts, displs, MPI_DOUBLE, local_A, my_local_n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    for (long i = 0; i < my_local_n; i++)
        local_sum += local_A[i];

    MPI_Reduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    end_time = MPI_Wtime();
    parallel_time = end_time - start_time;

    MPI_Allreduce(&local_sum, &avg_sum, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
    avg_sum /= N;

    if (rank == 0) {
        double expected = (N * (N + 1)) / 2.0;

        double serial_sum = 0.0;
        double serial_start = MPI_Wtime();
        for (long i = 0; i < N; i++)
            serial_sum += A[i];
        double serial_end = MPI_Wtime();
        serial_time = serial_end - serial_start;

        printf("\n===== Distributed Partial Summation using MPI =====\n");
        printf("Vector Size (N): %ld\n", N);
        printf("Processes Used: %d\n", size);
        printf("Computed Total Sum = %.0f\n", global_sum);
        printf("Expected Total Sum = %.0f\n", expected);
        printf("Difference = %.5f\n", expected - global_sum);
        printf("Average (MPI_Allreduce) = %.5f\n", avg_sum);
        printf("Parallel Execution Time: %.6f seconds\n", parallel_time);
        printf("Serial Execution Time: %.6f seconds\n", serial_time);
        printf("Speedup = %.2fx\n", serial_time / parallel_time);
        printf("====================================================\n");

        free(A);
        free(sendcounts);
        free(displs);
    }

    free(local_A);
    MPI_Finalize();
    return 0;
}


Writing distributed_sum_full.c


In [18]:
!mpicc distributed_sum.c -o distributed_sum


In [19]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./distributed_sum



===== Distributed Partial Summation using MPI =====
Vector Size (N): 10000000
Processes Used: 4
Computed Total Sum: 50000005000000
Expected Total Sum: 50000005000000
Difference: 0.00000
Average: 5000000.50000
Execution Time: 0.101384 seconds


### Discussion Questions and Answers

1. What happens if N is not divisible by the number of processes?  
If N is not evenly divisible, some processes would receive fewer elements, and some data might be skipped. This program fixes that by using variable send counts in MPI_Scatterv.

2. How can you modify the program to handle uneven partitions?  
Compute arrays of send counts and displacements (sendcounts[] and displs[]) and use MPI_Scatterv instead of MPI_Scatter to ensure each process receives the correct number of elements.

3. How would performance differ between using MPI_Reduce versus MPI_Gather plus local summation?  
MPI_Reduce performs a distributed sum within the MPI library, which is faster and more memory efficient. MPI_Gather collects all results at the root and sums them serially, which is slower and requires more memory.

4. How could this same approach be extended to matrix summation or averaging?  
Treat each row or block of rows as a subarray, distribute them with MPI_Scatterv, compute local partial sums, and then use MPI_Reduce or MPI_Allreduce for the total or average.

### Bonus Challenge
This program already:
- Computes both sum and average using a single collective operation (MPI_Allreduce).  
- Measures execution time with MPI_Wtime.  
- Compares parallel and serial CPU computation to show speedup.
